In [1]:
import numpy as np
import pandas as pd

import librosa

import os
PATH = os.getenv('AUDIO_PATH') # Folder where FMA data is unzipped

This notebook covers the process of extracting the relevant track info along with computing our mel-spectograms for each song and storing them for use in later notebooks. An explanation of what a mel-spectogram is and why it is necessary can be found in the [EDA](./02-EDA.ipynb) notebook.

In [2]:
# Specify the exact directory mp3s are housed in
audio_PATH = PATH + "fma_small/"


# Create lists to hold our data
audio = []
track_id = []


# These 6 files need to be skipped due to incomplete data 
# See: https://github.com/mdeff/fma/wiki#excerpts-shorter-than-30s-and-erroneous-audio-length-metadata
bad_files = [
    '098/098565.mp3',
    '098/098567.mp3',
    '098/098569.mp3',
    '099/099134.mp3',
    '108/108925.mp3',
    '133/133297.mp3'
]

# Iterate through each item in the directory
for item in os.listdir(audio_PATH):
    # Check that the item isn't a metadata file
    if 'check' not in item and '.txt' not in item:
        # Iterate through contents of the folder
        for file in os.listdir(audio_PATH + item):
            # Skip incomplete songs
            if item + '/' + file in bad_files:
                continue
            # Create local filepath and grab track id
            track_id.append(item + '/' + file)
            file_path = audio_PATH + track_id[-1]

            # Read song in using librosa
            song, sr = librosa.load(file_path)

            # Compute then rescale melspectogram into decibels
            mel = librosa.power_to_db(librosa.feature.melspectrogram(y = song, sr = sr)).T[:1291]

            # Append mel-spectogram into list
            audio.append(mel)

In [3]:
# Create dataframe for raw_audio data
raw_data = pd.Series(
    audio, 
    index = pd.Series(
        [int(id[-10:-4]) for id in track_id]
        ), 
    name = 'audio')

In [4]:
# Read in track data
# Header is necessary due to the structure of the CSV
df = pd.read_csv(PATH + "fma_metadata/tracks.csv", index_col=0, header=[0, 1])

The tracks file provided is composed of multiple dataframes which need to be called and handled separately. We chose to sequentially merge our data into exactly the info we want for our analysis.

In [5]:
# Merge the relevant parts of each sub-dataframe
artist_album = pd.merge(
    df['artist'],
    df['album'],
    how = 'inner',
    left_index = True,
    right_index = True
).rename(
    {
        'name' : 'name',
        'title' : 'album'
    },
    axis = 1
)[['name', 'album']] # Only need artist name and album here


# Gather only the songs in the 'small' subset
# 'set' here refers to the pre-defined train-test splits
genre_artists = pd.merge(
    df['set'][df['set']['subset'] == 'small'],
    artist_album,
    how = 'left',
    left_index = True,
    right_index = True
)

# Merge artist data back into the track info 
# Select only relevant columns
tracks = pd.merge(
    df['track'],
    genre_artists,
    how = 'inner',
    left_index = True,
    right_index = True
)[['title', 'name', 'album', 'genre_top', 'split']]


# Filter out incomplete rows and export to csv
indices = [int(file[-10:-4]) for file in bad_files]
tracks.loc[~tracks.index.isin(indices), ].to_csv('../data/tracks_meta.csv')

In [6]:
# Save the individual mel-spectogram for each song
# Using numpy's save feature because our data has issues being stored as a csv through pandas
for idx, data in zip(raw_data.index, raw_data.values):
    np.save('../data/audio' + str(idx), data)